In [77]:
from typing import Dict, List, Optional, Tuple
from enum import Enum
from pydantic import BaseModel, field_validator, ConfigDict, Field
from supabase import create_client
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import logging
import yfinance as yf
import os
import pandas as pd
import time

model_config = ConfigDict(
    extra="ignore",  # Allow extra fields in YFinance responses
    arbitrary_types_allowed=True,  # Allow pandas types
)


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
supabase_client = create_client("https://wwzyziohidsxmjfkoowh.supabase.co", "[REDACTED_JWT]")

In [15]:
event = {
    
    
}

In [16]:
class Ticker(BaseModel):
    id: str
    symbol: str
    exchange: str
    backfill: bool
    market_last_updated_at: Optional[str]
        
class EventType(str, Enum):
    """Types of events that can be processed by the pipeline."""

    SCHEDULED = "scheduled"  # Regular scheduled update
    INITIALIZE = "initialize"  # Process new tickers with backfill
    UPDATE = "update"  # Force update specific tickers
    BACKFILL = "backfill"  # Historical data backfill

class PipelineConfig(BaseModel):
    process_prices: bool = True
    process_info: bool = True
    process_calendar: bool = True
    process_fund_data: bool = True
    force_update: bool = False
    backfill: bool = False
    start_date: Optional[date] = None
    specific_tickers: Optional[List[str]] = None
        
class EventConfig(BaseModel):
    """Configuration options that can be provided in an event."""

    backfill: Optional[bool] = None
    prices: Optional[bool] = None
    info: Optional[bool] = None
    calendar: Optional[bool] = None
    fund_data: Optional[bool] = None


        
class EventPayload(BaseModel):
    """Schema for the event payload."""

    type: EventType = EventType.SCHEDULED
    tickers: Optional[List[str]] = None
    start_date: Optional[str] = None
    config: Optional[EventConfig] = None

    @field_validator("start_date")
    @classmethod
    def validate_start_date(cls, v):
        """Validate and convert start_date to the correct format."""
        if v is None:
            return None
        try:
            # Parse date string in YYYY-MM-DD format
            parsed_date = datetime.strptime(v, "%Y-%m-%d").date()
            return parsed_date
        except ValueError:
            raise ValueError("start_date must be in YYYY-MM-DD format")
            
            
# YH Finance Models




In [38]:

class YFQuoteType(str, Enum):
    """Quote types from YFinance"""

    EQUITY = "EQUITY"
    ETF = "ETF"
    MUTUALFUND = "MUTUALFUND"
    INDEX = "INDEX"
    CURRENCY = "CURRENCY"
    CRYPTOCURRENCY = "CRYPTOCURRENCY"

class YFTickerInfo(BaseModel):
    """Ticker information from YFinance"""

    model_config = model_config

    # Basic information
    symbol: str
    short_name: Optional[str] = Field(None, alias="shortName")
    long_name: Optional[str] = Field(None, alias="longName")
    quote_type: Optional[YFQuoteType] = Field(None, alias="quoteType")
    exchange: Optional[str] = None
    currency: Optional[str] = None

    # Business information
    sector: Optional[str] = None
    industry: Optional[str] = None
    full_time_employees: Optional[int] = Field(None, alias="fullTimeEmployees")
    long_business_summary: Optional[str] = Field(None, alias="longBusinessSummary")

    # Market data
    regular_market_price: Optional[float] = Field(None, alias="regularMarketPrice")
    regular_market_open: Optional[float] = Field(None, alias="regularMarketOpen")
    regular_market_day_high: Optional[float] = Field(None, alias="regularMarketDayHigh")
    regular_market_day_low: Optional[float] = Field(None, alias="regularMarketDayLow")
    regular_market_volume: Optional[int] = Field(None, alias="regularMarketVolume")
    market_cap: Optional[int] = Field(None, alias="marketCap")
    regular_market_change_percent: Optional[float] = Field(
        None, alias="regularMarketChangePercent"
    )

    # Technical indicators
    fifty_two_week_low: Optional[float] = Field(None, alias="fiftyTwoWeekLow")
    fifty_two_week_high: Optional[float] = Field(None, alias="fiftyTwoWeekHigh")
    fifty_day_average: Optional[float] = Field(None, alias="fiftyDayAverage")
    two_hundred_day_average: Optional[float] = Field(None, alias="twoHundredDayAverage")

    # Fundamentals - Stocks
    trailing_pe: Optional[float] = Field(None, alias="trailingPE")
    forward_pe: Optional[float] = Field(None, alias="forwardPE")
    dividend_yield: Optional[float] = Field(None, alias="dividendYield")
    dividend_rate: Optional[float] = Field(None, alias="dividendRate")
    beta: Optional[float] = None

    # Fundamentals - Funds
    nav_price: Optional[float] = Field(None, alias="navPrice")
    yield_value: Optional[float] = Field(None, alias="yield")
    ytd_return: Optional[float] = Field(None, alias="ytdReturn")
    total_assets: Optional[int] = Field(None, alias="totalAssets")
    fund_family: Optional[str] = Field(None, alias="fundFamily")
    fund_inception_date: Optional[float] = Field(
        None, alias="fundInceptionDate"
    )  # Unix timestamp
    legal_type: Optional[str] = Field(None, alias="legalType")
    three_year_average_return: Optional[float] = Field(
        None, alias="threeYearAverageReturn"
    )
    five_year_average_return: Optional[float] = Field(
        None, alias="fiveYearAverageReturn"
    )
    net_expense_ratio: Optional[float] = Field(None, alias="netExpenseRatio")

    # Other fields
    last_dividend_value: Optional[float] = Field(None, alias="lastDividendValue")
    last_dividend_date: Optional[float] = Field(
        None, alias="lastDividendDate"
    )  # Unix timestamp
    shares_outstanding: Optional[int] = Field(None, alias="sharesOutstanding")

    def get_fund_inception_date(self) -> Optional[date]:
        """Convert Unix timestamp to date object"""
        if self.fund_inception_date is not None:
            try:
                return datetime.fromtimestamp(self.fund_inception_date).date()
            except (ValueError, TypeError, OverflowError):
                pass
        return None



In [17]:
class TickerFetcher:
    def __init__(self, supabase_client):
        self.supabase = supabase_client

    def fetch_tickers(self):
        try:
            response = (
                self.supabase.table("tickers")
                .select("id, symbol, exchange, backfill, market_last_updated_at")
                .execute()
            )
            return response.data
        except Exception as e:
            return []


In [18]:
tickerFetcher = TickerFetcher(supabase_client)

In [19]:
db_tickers = tickerFetcher.fetch_tickers()

In [20]:
tickers = [Ticker(**ticker) for ticker in db_tickers]


In [43]:
tech = yf.Sector('technology').overview

In [44]:
tech

{'companies_count': 821,
 'market_cap': 16515721068544,
 'message_board_id': 'INDEXYH311',
 'description': 'Companies engaged in the design, development, and support of computer operating systems and applications. This sector also includes companies that make computer equipment, data storage products, networking products, semiconductors, and components. Companies in this sector include Apple, Microsoft, and IBM.',
 'industries_count': 12,
 'market_weight': 0.27559972,
 'employee_count': 7263724}

In [39]:
datetime.utcnow()


datetime.datetime(2025, 4, 11, 20, 48, 30, 852657)

In [63]:
class SectorInfo(BaseModel):
    key: str
    name: str
    description: Optional[str] = None
    companies_count: Optional[int] = None
    market_cap: Optional[int] = None
    employee_count: Optional[int] = None
    market_weight: Optional[float] = None
    updated_at: str = Field(default_factory=lambda: datetime.now().isoformat())

class IndustryInfo(BaseModel):
    key: str
    name: str
    sector_id: str
    description: Optional[str] = None
    companies_count: Optional[int] = None
    market_cap: Optional[int] = None
    employee_count: Optional[int] = None
    market_weight: Optional[float] = None
    updated_at: str = Field(default_factory=lambda: datetime.now().isoformat())

class IndexTickerInfo(BaseModel):
    symbol: str
    name: str
    quote_type: str = "INDEX"
    tradeable: bool = False
    currency: Optional[str] = None
    exchange: Optional[str] = None
    region: Optional[str] = None
    sector_id: Optional[str] = None
    industry_id: Optional[str] = None
    updated_at: str = Field(default_factory=lambda: datetime.now().isoformat())

sector_keys = [
    "basic-materials", "communication-services", "consumer-cyclical", 
    "consumer-defensive", "energy", "financial-services", "healthcare", 
    "industrials", "real-estate", "technology", "utilities"
]

In [64]:
def process_industry(industry_key: str, industry_name: str, sector_id: str, supabase_client) -> Optional[str]:
    try:
        current_time = datetime.utcnow().isoformat()
        # Prepare the update payload with default values
        payload = {
            "name": industry_name,
            "updated_at": current_time
        }
        # Enrich payload from yfinance if data is available
        try:
            industry_obj = yf.Industry(industry_key)
            overview = industry_obj.overview
            if isinstance(overview, dict):
                payload.update({
                    "description": overview.get("description"),
                    "companies_count": overview.get("companies_count"),
                    "market_cap": overview.get("market_cap"),
                    "employee_count": overview.get("employee_count"),
                    "market_weight": overview.get("market_weight")
                })
        except Exception as e:
            logger.warning(f"Could not fetch additional data for industry {industry_key}: {e}")

        # Query to check if the industry exists
        existing = supabase_client.table("industries") \
                                  .select("id") \
                                  .eq("key", industry_key) \
                                  .eq("sector_id", sector_id) \
                                  .execute()
        if not existing.data or len(existing.data) == 0:
            logger.error(f"Industry {industry_key} (sector: {sector_id}) not found. Skipping update.")
            return None

        industry_id = existing.data[0]["id"]
        update_result = supabase_client.table("industries") \
                                       .update(payload) \
                                       .eq("id", industry_id) \
                                       .execute()
        if not update_result.data:
            logger.error(f"Failed to update industry {industry_key}")
            return None

        logger.info(f"Updated industry: {industry_name}")
        return industry_id

    except Exception as e:
        logger.error(f"Error processing industry {industry_key}: {e}")
        return None

# ---------------------------------------------------------------------------
# Function to update existing sectors and their industries only
# ---------------------------------------------------------------------------
def populate_sectors_and_industries(supabase_client):
    sectors_updated = 0
    industries_updated = 0

    for sector_key in sector_keys:
        try:
            logger.info(f"Processing sector: {sector_key}")
            sector = yf.Sector(sector_key)
            current_time = datetime.utcnow().isoformat()
            overview = sector.overview

            payload = {
                "name": sector.name,
                "updated_at": current_time,
                "description": overview.get("description") if isinstance(overview, dict) else getattr(overview, "description", None),
                "companies_count": overview.get("companies_count") if isinstance(overview, dict) else getattr(overview, "companies_count", None),
                "market_cap": overview.get("market_cap") if isinstance(overview, dict) else getattr(overview, "market_cap", None),
                "employee_count": overview.get("employee_count") if isinstance(overview, dict) else getattr(overview, "employee_count", None),
                "market_weight": overview.get("market_weight") if isinstance(overview, dict) else getattr(overview, "market_weight", None)
            }

            # Query to check if the sector exists
            existing_sector = supabase_client.table("sectors") \
                                             .select("id") \
                                             .eq("key", sector_key) \
                                             .execute()
            if not existing_sector.data or len(existing_sector.data) == 0:
                logger.error(f"Sector {sector_key} not found. Skipping update.")
                continue

            sector_id = existing_sector.data[0]["id"]
            update_result = supabase_client.table("sectors") \
                                           .update(payload) \
                                           .eq("id", sector_id) \
                                           .execute()
            if not update_result.data:
                logger.error(f"Failed to update sector {sector_key}")
                continue

            sectors_updated += 1
            logger.info(f"Updated sector: {sector.name}")

            # Process industries for this sector
            industry_list = []
            if hasattr(sector, 'industries'):
                # Handle if industries is a DataFrame, dictionary, or list.
                for index, row in sector.industries.iterrows():
                    industry_list.append((index, row.get('name', index)))

            # Update each industry belonging to the sector
            for industry_key, industry_name in industry_list:
                industry_id = process_industry(industry_key, industry_name, sector_id, supabase_client)
                if industry_id:
                    industries_updated += 1

        except Exception as e:
            logger.error(f"Error processing sector {sector_key}: {e}")

    logger.info(f"Update complete! Updated {sectors_updated} sectors and {industries_updated} industries.")
    return sectors_updated, industries_updated

In [65]:
populate_sectors_and_industries(supabase_client)

INFO:__main__:Processing sector: basic-materials
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?select=id&key=eq.basic-materials "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?id=eq.e2ce1a69-cdcb-4368-a7ff-f05507b9b6ff "HTTP/2 200 OK"
INFO:__main__:Updated sector: Basic Materials
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.specialty-chemicals&sector_id=eq.e2ce1a69-cdcb-4368-a7ff-f05507b9b6ff "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.793f1e5b-5eb7-40fd-91ea-6c277a79a9f9 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Specialty Chemicals
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.gold&sector_id=eq.e2ce1a69-cdcb-4368-a7ff-f05507b9b6ff "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxm

INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.c8f7f77d-f3f6-42f9-9c1b-96ceae640f3e "HTTP/2 200 OK"
INFO:__main__:Updated industry: Broadcasting
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.publishing&sector_id=eq.9570ba22-3ae9-47ca-9e2b-0e0e3401cbe1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.fdc38861-4e4b-4600-b18a-10b52682c239 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Publishing
INFO:__main__:Processing sector: consumer-cyclical
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?select=id&key=eq.consumer-cyclical "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?id=eq.f1a1968b-1d59-403a-8395-fbdf2e8ec07e "HTTP/2 200 OK"
INFO:__main__:Updated sector: Consumer Cyclical
INFO:httpx:HTTP Request: GET https://wwzyziohi

INFO:__main__:Updated industry: Apparel Manufacturing
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.luxury-goods&sector_id=eq.f1a1968b-1d59-403a-8395-fbdf2e8ec07e "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.b83eed72-8c28-42dd-b19d-b4466b8d4e66 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Luxury Goods
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.recreational-vehicles&sector_id=eq.f1a1968b-1d59-403a-8395-fbdf2e8ec07e "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.ebc1d2bf-31af-4958-86a0-1d1227d2e450 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Recreational Vehicles
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.department-stores&sector_id=eq.f1a1968b-1d59-403a-8395-fbdf2e8

INFO:__main__:Updated industry: Oil & Gas E&P
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.oil-gas-equipment-services&sector_id=eq.7c7419eb-1492-48bd-8395-bc9fe9132319 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.8d0edca7-eb10-4ac0-b04e-9d0ad85276e4 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Oil & Gas Equipment & Services
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.oil-gas-refining-marketing&sector_id=eq.7c7419eb-1492-48bd-8395-bc9fe9132319 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.93cf2e03-9689-42fd-9f27-f75ef19d2b17 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Oil & Gas Refining & Marketing
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.uranium&sector_id=eq.7c7

INFO:__main__:Updated industry: Shell Companies
INFO:__main__:Processing sector: healthcare
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?select=id&key=eq.healthcare "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?id=eq.c49c8f48-dbbd-4204-a679-9f53f9a077d3 "HTTP/2 200 OK"
INFO:__main__:Updated sector: Healthcare
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.drug-manufacturers-general&sector_id=eq.c49c8f48-dbbd-4204-a679-9f53f9a077d3 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.cced79ea-fd47-46ce-838f-22c1f2cd395f "HTTP/2 200 OK"
INFO:__main__:Updated industry: Drug Manufacturers - General
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.healthcare-plans&sector_id=eq.c49c8f48-dbbd-4204-a679-9f53f9a077d3 "HTTP/2

INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.1a9ab0ed-fdc4-497c-8572-111828f319ba "HTTP/2 200 OK"
INFO:__main__:Updated industry: Conglomerates
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.industrial-distribution&sector_id=eq.153d43ac-db19-476b-9f9d-5f8da92e2524 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.f1831584-fc6d-4132-b819-c4f6e22e138b "HTTP/2 200 OK"
INFO:__main__:Updated industry: Industrial Distribution
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.engineering-construction&sector_id=eq.153d43ac-db19-476b-9f9d-5f8da92e2524 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.5e9f279b-1922-4b90-b6f1-36cd4d92ba58 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Engineering & Construct

INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.2f074474-53f3-40ea-9b7b-2acf9eadf1b9 "HTTP/2 200 OK"
INFO:__main__:Updated industry: REIT - Retail
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.reit-residential&sector_id=eq.f813c528-a29d-467f-85e7-3e16851ba5b4 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.5b118d1a-74b0-40f6-b74a-83b2103b35fd "HTTP/2 200 OK"
INFO:__main__:Updated industry: REIT - Residential
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.reit-healthcare-facilities&sector_id=eq.f813c528-a29d-467f-85e7-3e16851ba5b4 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.477504b3-26fd-4414-9672-067f569d2fef "HTTP/2 200 OK"
INFO:__main__:Updated industry: REIT - Healthcare Facilities
INFO

INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.64a1557e-aee4-4c70-acd8-5bc22ddef55b "HTTP/2 200 OK"
INFO:__main__:Updated industry: Solar
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id&key=eq.electronics-computer-distribution&sector_id=eq.bbb9d123-6162-461e-8144-3467fd21da25 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?id=eq.9d326d48-0ed8-4ea6-a04d-ab274bb81f40 "HTTP/2 200 OK"
INFO:__main__:Updated industry: Electronics & Computer Distribution
INFO:__main__:Processing sector: utilities
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?select=id&key=eq.utilities "HTTP/2 200 OK"
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?id=eq.af344b06-f006-4760-ad92-cea3754a15b3 "HTTP/2 200 OK"
INFO:__main__:Updated sector: Utilities
INFO:httpx:HTTP Request: GET 

(11, 145)

In [18]:
# Now let's update old sectors
START_DATE = date(2020, 1, 1)
END_DATE = date.today()
BATCH_SIZE = 5  # Number of tickers to process in one batch
RATE_LIMIT_DELAY = 2  # Delay between individual API calls (seconds)
BATCH_DELAY = 30


In [19]:
def get_index_tickers() -> List[Dict]:
    """Get all INDEX tickers from the database"""
    try:
        response = supabase_client.table("tickers").select("id, symbol, name").eq("quote_type", "INDEX").execute()
        return response.data
    except Exception as e:
        logger.error(f"Error fetching INDEX tickers: {e}")
        return []

In [22]:
def save_historical_prices(ticker_id: str, symbol: str, price_data: pd.DataFrame) -> int:
    """Save historical price data to the database"""
    if price_data is None or price_data.empty:
        logger.warning(f"No price data for {symbol}")
        return 0

    try:
        # Prepare data for insertion
        price_records = []
        for index, row in price_data.iterrows():
            date_str = index.strftime("%Y-%m-%d")
            price_records.append({
                "ticker_id": ticker_id,
                "date": date_str,
                "open_price": float(row["Open"]) if not pd.isna(row["Open"]) else None,
                "high_price": float(row["High"]) if not pd.isna(row["High"]) else None,
                "low_price": float(row["Low"]) if not pd.isna(row["Low"]) else None,
                "close_price": float(row["Close"]) if not pd.isna(row["Close"]) else None,
                "volume": int(row["Volume"]) if not pd.isna(row["Volume"]) else None,
                "dividends": float(row["Dividends"]) if "Dividends" in row and not pd.isna(row["Dividends"]) else None,
                "stock_splits": float(row["Stock Splits"]) if "Stock Splits" in row and not pd.isna(row["Stock Splits"]) else None,
                "updated_at": datetime.now().isoformat()
            })

        # Insert data in chunks to avoid hitting limitations
        chunk_size = 100
        for i in range(0, len(price_records), chunk_size):
            chunk = price_records[i:i+chunk_size]
            # Use the constraint name explicitly
            supabase_client.table("historical_prices").upsert(
                chunk, 
                on_conflict="ticker_id, date"
            ).execute()
            # Small delay to avoid rate limiting
            time.sleep(0.1)

        logger.info(f"Saved {len(price_records)} price records for {symbol}")
        return len(price_records)
    except Exception as e:
        logger.error(f"Error saving price data for {symbol}: {e}")
        return 0

def process_index_ticker(ticker: Dict) -> Tuple[bool, int]:
    """Process a single INDEX ticker to backfill historical data"""
    ticker_id = ticker["id"]
    symbol = ticker["symbol"]
    
    logger.info(f"Processing index: {symbol} ({ticker_id})")
    
    try:
        # Check for existing data to avoid duplicate work
        existing_response = supabase_client.table("historical_prices") \
            .select("date") \
            .eq("ticker_id", ticker_id) \
            .order("date", desc=True) \
            .limit(1) \
            .execute()
            
        start_date = START_DATE
        
        if existing_response.data and len(existing_response.data) > 0:
            latest_date = datetime.strptime(existing_response.data[0]["date"], "%Y-%m-%d").date()
            # If we already have recent data, skip this ticker
            if latest_date >= END_DATE - timedelta(days=7):
                logger.info(f"Skipping {symbol} - data is up to date")
                return True, 0
            # Otherwise, start from the day after the latest date
            start_date = latest_date + timedelta(days=1)
            logger.info(f"Starting from {start_date} for {symbol}")
        
        # Fetch historical data from Yahoo Finance
        yf_ticker = yf.Ticker(symbol)
        historical_data = yf_ticker.history(start=start_date, end=END_DATE, auto_adjust=True)
        
        if historical_data.empty:
            logger.warning(f"No historical data available for {symbol}")
            return False, 0
            
        # Save the data
        saved_count = save_historical_prices(ticker_id, symbol, historical_data)
        return saved_count > 0, saved_count
        
    except Exception as e:
        logger.error(f"Error processing {symbol}: {e}")
        return False, 0

def backfill_index_data():
    """Main function to backfill historical data for all INDEX tickers"""
    logger.info("Starting INDEX historical data backfill")
    
    # Get all INDEX tickers
    index_tickers = get_index_tickers()
    logger.info(f"Found {len(index_tickers)} INDEX tickers to process")
    
    if not index_tickers:
        logger.warning("No INDEX tickers found. Check your database or quote_type values.")
        return
    
    # Process tickers in batches
    successful = 0
    failed = 0
    total_records = 0
    
    for i in range(0, len(index_tickers), BATCH_SIZE):
        batch = index_tickers[i:i+BATCH_SIZE]
        logger.info(f"Processing batch {i//BATCH_SIZE + 1}/{(len(index_tickers) + BATCH_SIZE - 1)//BATCH_SIZE}")
        
        for ticker in batch:
            success, count = process_index_ticker(ticker)
            if success:
                successful += 1
                total_records += count
            else:
                failed += 1
            
            # Sleep between API calls to avoid rate limits
            time.sleep(RATE_LIMIT_DELAY)
        
        # Sleep between batches
        if i + BATCH_SIZE < len(index_tickers):
            logger.info(f"Batch complete. Sleeping for {BATCH_DELAY} seconds before next batch...")
            time.sleep(BATCH_DELAY)
    
    logger.info(f"Backfill complete. Processed {len(index_tickers)} tickers: {successful} successful, {failed} failed.")
    logger.info(f"Total records saved: {total_records}")


In [23]:
backfill_index_data()

INFO:__main__:Starting INDEX historical data backfill
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/tickers?select=id%2Csymbol%2Cname&quote_type=eq.INDEX "HTTP/2 200 OK"
INFO:__main__:Found 156 INDEX tickers to process
INFO:__main__:Processing batch 1/32
INFO:__main__:Processing index: ^YH206 (cb05f43c-fa16-4648-a8f5-b710e1c23b39)
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?select=date&ticker_id=eq.cb05f43c-fa16-4648-a8f5-b710e1c23b39&order=date.desc&limit=1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_c

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:__main__:Saved 1327 price records for ^YH207
INFO:__main__:Batch complete. Sleeping for 30 seconds before next batch...
INFO:__main__:Processing batch 2/32
INFO:__main__:Processing index: ^YH101 (5de0521c-5965-4ef3-9661-ff1428e5e6fa)
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?select=date&ticker_id=eq.5de0521c-5965-4ef3-9661-ff1428e5e6fa&order=date.desc&limit=1 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%2

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

...

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 Created"
INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/historical_prices?on_conflict=ticker_id%2C%20date&columns=%22dividends%22%2C%22date%22%2C%22open_price%22%2C%22high_price%22%2C%22updated_at%22%2C%22stock_splits%22%2C%22ticker_id%22%2C%22low_price%22%2C%22volume%22%2C%22close_price%22 "HTTP/2 201 C

In [39]:
# Migrating old sector mappings etc..
# Settings
BATCH_SIZE = 100
BATCH_DELAY = 2  # seconds

def get_sector_mapping() -> Dict[str, str]:
    """Get mapping of sector names to sector IDs"""
    try:
        response = supabase_client.table("sectors").select("id, name, key").execute()
        
        # Create two mappings - by name and by key
        mapping_by_name = {sector["name"].lower(): sector["id"] for sector in response.data}
        mapping_by_key = {sector["key"].lower(): sector["id"] for sector in response.data}
        
        # Combine mappings, prioritizing key mapping
        combined_mapping = {**mapping_by_name, **mapping_by_key}
        
        logger.info(f"Found {len(combined_mapping)} sector mappings")
        return combined_mapping
    except Exception as e:
        logger.error(f"Error getting sector mapping: {e}")
        return {}

def get_industry_mapping() -> Dict[str, Dict[str, str]]:
    """Get mapping of industry names to industry IDs with their sector IDs"""
    try:
        response = supabase_client.table("industries").select("id, name, key, sector_id").execute()
        
        # Create two mappings - by name and by key
        mapping_by_name = {
            industry["name"].lower(): {"id": industry["id"], "sector_id": industry["sector_id"]} 
            for industry in response.data
        }
        mapping_by_key = {
            industry["key"].lower(): {"id": industry["id"], "sector_id": industry["sector_id"]} 
            for industry in response.data
        }
        
        # Combine mappings, prioritizing key mapping
        combined_mapping = {**mapping_by_name, **mapping_by_key}
        
        logger.info(f"Found {len(combined_mapping)} industry mappings")
        return combined_mapping
    except Exception as e:
        logger.error(f"Error getting industry mapping: {e}")
        return {}

def get_tickers_needing_update() -> List[Dict]:
    """Get tickers that need sector_id or industry_id update"""
    try:
        response = (supabase_client.table("tickers")
            .select("id, symbol, name, sector, industry, sector_id, industry_id, currency")
            .or_("sector_id.is.null,industry_id.is.null,currency.is.null")
            .filter("sector", "neq", None)
            .execute())
        
        return response.data
    except Exception as e:
        logger.error(f"Error fetching tickers to update: {e}")
        return []

def update_ticker(
    ticker: Dict, 
    sector_mapping: Dict[str, str], 
    industry_mapping: Dict[str, Dict[str, str]]
) -> bool:
    """Update a single ticker with new relationship IDs"""
    ticker_id = ticker["id"]
    symbol = ticker["symbol"]
    update_data = {}
    
    # Set default currency if missing
    if not ticker.get("currency"):
        # Default to USD for US exchanges, otherwise leave it blank for now
        if ticker.get("exchange") in ["NYSE", "NASDAQ", "AMEX", None]:
            update_data["currency"] = "USD"
    
    # Find sector_id if missing
    if not ticker.get("sector_id") and ticker.get("sector"):
        sector_name = ticker["sector"].lower()
        sector_id = sector_mapping.get(sector_name)
        
        if sector_id:
            update_data["sector_id"] = sector_id
            logger.debug(f"Matched sector '{ticker['sector']}' to ID {sector_id}")
        else:
            logger.warning(f"Could not find sector ID for '{ticker['sector']}' (ticker: {symbol})")
    
    # Find industry_id if missing
    if not ticker.get("industry_id") and ticker.get("industry"):
        industry_name = ticker["industry"].lower()
        industry_info = industry_mapping.get(industry_name)
        
        if industry_info:
            update_data["industry_id"] = industry_info["id"]
            
            # If we don't have a sector_id yet, use the one from the industry
            if not update_data.get("sector_id") and not ticker.get("sector_id"):
                update_data["sector_id"] = industry_info["sector_id"]
                
            logger.debug(f"Matched industry '{ticker['industry']}' to ID {industry_info['id']}")
        else:
            logger.warning(f"Could not find industry ID for '{ticker['industry']}' (ticker: {symbol})")
    
    # Only update if we have changes to make
    if update_data:
        try:
            update_data["updated_at"] = datetime.now().isoformat()
            supabase_client.table("tickers").update(update_data).eq("id", ticker_id).execute()
            logger.info(f"Updated ticker {symbol} ({ticker_id}) with {', '.join(update_data.keys())}")
            return True
        except Exception as e:
            logger.error(f"Error updating ticker {symbol}: {e}")
            return False
    else:
        logger.debug(f"No updates needed for ticker {symbol}")
        return False

def migrate_ticker_relationships():
    """Main function to migrate tickers to use the new relationships"""
    logger.info("Starting ticker relationship migration")
    
    # Get mappings
    sector_mapping = get_sector_mapping()
    industry_mapping = get_industry_mapping()
    
    if not sector_mapping or not industry_mapping:
        logger.error("Missing sector or industry mappings. Make sure these tables are populated.")
        return
    
    # Get tickers needing updates
    tickers = get_tickers_needing_update()
    logger.info(f"Found {len(tickers)} tickers that need updating")
    
    if not tickers:
        logger.info("No tickers need updating. Migration complete.")
        return
    
    # Process tickers in batches
    successful = 0
    failed = 0
    
    for i in range(0, len(tickers), BATCH_SIZE):
        batch = tickers[i:i+BATCH_SIZE]
        logger.info(f"Processing batch {i//BATCH_SIZE + 1}/{(len(tickers) + BATCH_SIZE - 1)//BATCH_SIZE}")
        
        for ticker in batch:
            if update_ticker(ticker, sector_mapping, industry_mapping):
                successful += 1
            else:
                failed += 1
        
        # Sleep between batches
        if i + BATCH_SIZE < len(tickers):
            logger.info(f"Batch complete. Sleeping for {BATCH_DELAY} seconds...")
            time.sleep(BATCH_DELAY)
    
    logger.info(f"Migration complete. Updated {successful} tickers successfully, {failed} failed.")
    
    # Output stats on remaining unlinked tickers
    try:
        unlinked_response = (
            supabase_client.table("tickers")
            .select("count")
            .or_("sector_id.is.null,industry_id.is.null")
            .execute()
        )
        
        remaining_count = unlinked_response.count
        logger.info(f"Tickers still without sector or industry relationships: {remaining_count}")
        
        if remaining_count > 0:
            logger.info("Consider running this script again or manually updating the remaining tickers.")
    except Exception as e:
        logger.error(f"Error checking remaining unlinked tickers: {e}")


In [40]:
migrate_ticker_relationships()

INFO:__main__:Starting ticker relationship migration
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/sectors?select=id%2Cname%2Ckey "HTTP/2 200 OK"
INFO:__main__:Found 17 sector mappings
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/industries?select=id%2Cname%2Ckey%2Csector_id "HTTP/2 200 OK"
INFO:__main__:Found 267 industry mappings
INFO:httpx:HTTP Request: GET https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/tickers?select=id%2Csymbol%2Cname%2Csector%2Cindustry%2Csector_id%2Cindustry_id%2Ccurrency&or=%28sector_id.is.null%2Cindustry_id.is.null%2Ccurrency.is.null%29&sector=neq.None "HTTP/2 200 OK"
INFO:__main__:Found 9 tickers that need updating
INFO:__main__:Processing batch 1/1
INFO:httpx:HTTP Request: PATCH https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/tickers?id=eq.e26f0039-781c-468a-93ae-65835d8d742b "HTTP/2 200 OK"
INFO:__main__:Updated ticker SPYI (e26f0039-781c-468a-93ae-65835d8d742b) with currency, updated_at
IN

In [66]:
import yfinance as yf

EUROPE = yf.Market("EUROPE")

status = EUROPE.status
summary = EUROPE.summary

In [68]:
summary

{'FGI': {'language': 'en-US',
  'region': 'US',
  'quoteType': 'INDEX',
  'typeDisp': 'Index',
  'quoteSourceName': 'Delayed Quote',
  'triggerable': False,
  'customPriceAlertConfidence': 'LOW',
  'shortName': 'FTSE 100',
  'regularMarketChange': 50.930176,
  'regularMarketChangePercent': 0.6436063,
  'regularMarketTime': 1744385730,
  'regularMarketPrice': 7964.18,
  'regularMarketPreviousClose': 7913.25,
  'exchange': 'FGI',
  'market': 'gb_market',
  'fullExchangeName': 'FTSE Index',
  'marketState': 'CLOSED',
  'sourceInterval': 15,
  'exchangeDataDelayedBy': 15,
  'exchangeTimezoneName': 'Europe/London',
  'exchangeTimezoneShortName': 'BST',
  'gmtOffSetMilliseconds': 3600000,
  'esgPopulated': False,
  'tradeable': False,
  'cryptoTradeable': False,
  'hasPrePostMarketData': False,
  'firstTradeDateMilliseconds': 441964800000,
  'priceHint': 2,
  'symbol': '^FTSE'},
 'PAR': {'language': 'en-US',
  'region': 'US',
  'quoteType': 'INDEX',
  'typeDisp': 'Index',
  'quoteSourceName'

In [75]:
import requests

ALPHAVANTAGE_API_KEY="AQWUGWHMMH0Q0D9T"
url = 'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=JPY&apikey=demo'
r = requests.get(url)
data = r.json()

print(data)


{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'JPY', '4. To_Currency Name': 'Japanese Yen', '5. Exchange Rate': '143.49000000', '6. Last Refreshed': '2025-04-13 18:57:33', '7. Time Zone': 'UTC', '8. Bid Price': '143.48800000', '9. Ask Price': '143.49100000'}}


In [74]:
data

{'error': {'code': 'invalid_access_key',
  'message': 'You have not supplied a valid API Access Key.'}}

In [91]:
# FOREX STUFF

currency_pairs = {
    "USDEUR=X": ("USD", "EUR"),
    "USDGBP=X": ("USD", "GBP"),
    "USDJPY=X": ("USD", "JPY"),
    "USDCAD=X": ("USD", "CAD"),
    "USDAUD=X": ("USD", "AUD")
}

# Date range: 5 years
END_DATE = datetime(2025, 4, 13)
START_DATE = END_DATE - timedelta(days=5*365)  # Approx April 13, 2020

def fetch_historical_data(ticker: str, start: datetime, end: datetime) -> pd.DataFrame:
    """Fetch historical FOREX data using yfinance."""
    try:
        # Fetch daily data
        df = yf.download(ticker, start=start, end=end, interval="1d", progress=False)
        if df.empty:
            print(f"No data returned for {ticker}")
            return pd.DataFrame()
        return df[["Close"]].rename(columns={"Close": "rate"})

    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return pd.DataFrame()

def store_forex_data():
    """Fetch and store 5 years of historical FOREX data in Supabase."""
    for ticker, (base, target) in currency_pairs.items():
        print(f"Fetching data for {base}/{target} ({ticker})...")
        df = fetch_historical_data(ticker, START_DATE, END_DATE)

        if df.empty:
            continue
        
        # Prepare records for Supabase
        records = [
            {
                "base_currency": base,
                "target_currency": target,
                "rate": round(float(row["rate"]), 6),
                "date": index.strftime("%Y-%m-%d"),
            }
            for index, row in df.iterrows()
        ]
        
        # Batch upsert to avoid duplicates
        try:
            response = supabase_client.table("forex_rates").upsert(
                records,
                on_conflict="base_currency,target_currency,date"
            ).execute()
            print(f"Stored {len(records)} rates for {base}/{target}")
        except Exception as e:
            print(f"Error storing {base}/{target}: {e}")


In [92]:
store_forex_data()

Fetching data for USD/EUR (USDEUR=X)...


INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/forex_rates?on_conflict=base_currency%2Ctarget_currency%2Cdate&columns=%22date%22%2C%22target_currency%22%2C%22rate%22%2C%22base_currency%22 "HTTP/2 201 Created"


Stored 1303 rates for USD/EUR
Fetching data for USD/GBP (USDGBP=X)...


INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/forex_rates?on_conflict=base_currency%2Ctarget_currency%2Cdate&columns=%22date%22%2C%22target_currency%22%2C%22rate%22%2C%22base_currency%22 "HTTP/2 201 Created"


Stored 1303 rates for USD/GBP
Fetching data for USD/JPY (USDJPY=X)...


...

Stored 1303 rates for USD/JPY
Fetching data for USD/CAD (USDCAD=X)...


INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/forex_rates?on_conflict=base_currency%2Ctarget_currency%2Cdate&columns=%22date%22%2C%22target_currency%22%2C%22rate%22%2C%22base_currency%22 "HTTP/2 201 Created"


Stored 1303 rates for USD/CAD
Fetching data for USD/AUD (USDAUD=X)...


INFO:httpx:HTTP Request: POST https://wwzyziohidsxmjfkoowh.supabase.co/rest/v1/forex_rates?on_conflict=base_currency%2Ctarget_currency%2Cdate&columns=%22date%22%2C%22target_currency%22%2C%22rate%22%2C%22base_currency%22 "HTTP/2 201 Created"


Stored 1303 rates for USD/AUD


In [94]:
# Test for how much info we get about a sector


import yfinance as yf

tech = yf.Sector('technology')
software = yf.Industry('software-infrastructure')

# Common information
software.ticker.info
# tech.name
# tech.symbol
# tech.ticker
# tech.overview
# tech.research_reports

# # Sector information
# tech.industries

# # Industry information
# software.sector_key
# software.sector_name

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 5010300.0,
 'regularMarketPreviousClose': 5010300.0,
 'fiftyTwoWeekLow': 4055548.8,
 'fiftyTwoWeekHigh': 5368924.0,
 'fiftyDayAverage': 4636852.5,
 'twoHundredDayAverage': 4862299.0,
 'currency': 'USD',
 'tradeable': False,
 '52WeekChange': -2.7859628,
 'quoteType': 'INDEX',
 'symbol': '^YH31110030',
 'language': 'en-US',
 'region': 'US',
 'typeDisp': 'Index',
 'quoteSourceName': 'Delayed Quote',
 'triggerable': False,
 'customPriceAlertConfidence': 'LOW',
 'marketState': 'CLOSED',
 'fullExchangeName': 'YHD',
 'fiftyTwoWeekLowChange': 380251.75,
 'fiftyTwoWeekLowChangePercent': 0.09376086,
 'fiftyTwoWeekRange': '4055548.8 - 5368924.0',
 'fiftyTwoWeekHighChange': -933123.5,
 'fiftyTwoWeekHighChangePercent': -0.17380084,
 'fiftyTwoWeekChangePercent': -2.7859628,
 'fiftyDayAverageChange': -201052.0,
 'fiftyDayAverageChangePercent': -0.043359585,
 'twoHundredDayAverageChange': -426498.5,
 'twoHundredDayAverageChangePercent': -0.0877154,


In [100]:
# Testing getting German Equities

telekom = yf.Ticker("DTE.F")

In [101]:
telekom.info

{'address1': 'Friedrich-Ebert-Allee 140',
 'city': 'Bonn',
 'zip': '53113',
 'country': 'Germany',
 'phone': '49 228 181 88880',
 'website': 'https://www.telekom.com',
 'industry': 'Telecom Services',
 'industryKey': 'telecom-services',
 'industryDisp': 'Telecom Services',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Deutsche Telekom AG, together with its subsidiaries, provides integrated telecommunication services worldwide. The company operates through Germany, United States, Europe, Systems Solutions, and Group Development segments. It offers fixed-network services, including voice and data communication services based on fixed-network and broadband technology; and sells terminal equipment and other hardware products, as well as services to resellers. The company also provides mobile voice and data services to consumers and business customers; sells mobile devices and other hardware pro

In [102]:
spyi = yf.Ticker("SPYI")

In [103]:
spyi.info

{'phone': '9144435008',
 'longBusinessSummary': 'The fund is an actively-managed ETF that seeks to achieve its investment objective by investing in a portfolio of stocks that make up the S&P 500® Index and a call options strategy, which consists of a mix of written (sold) call options and long (bought) call options on the S&P 500® Index. Under certain circumstances, the call options strategy may include transactions with covered call options.',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 45.16,
 'open': 45.16,
 'dayLow': 44.812,
 'dayHigh': 46.065,
 'regularMarketPreviousClose': 45.16,
 'regularMarketOpen': 45.16,
 'regularMarketDayLow': 44.812,
 'regularMarketDayHigh': 46.065,
 'trailingPE': 23.688875,
 'volume': 980821,
 'regularMarketVolume': 980821,
 'averageVolume': 1181620,
 'averageVolume10days': 1836460,
 'averageDailyVolume10Day': 1836460,
 'bid': 46.0,
 'ask': 46.02,
 'bidSize': 8,
 'askSize': 8,
 'yield': 0.1288,
 'total